In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from numpy import sin, cos, arccos, pi, round

!pip install smogn
import smogn

from sklearn import neighbors
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

### 1° Leitura e tratamento inicial dos dados

Lendo a CSV e removendo todos os dados com valores de Tp_est iguais a zero

Resultados:
- Base de dados raw para treinamento (*df_raw_train*)
- Base de dados raw para validação (*df_raw_validation*)
- Base de dados raw para teste (*df_raw_test*)

In [3]:
df_raw_data = pd.read_csv('/content/drive/MyDrive/BCC - UFPR/Semestres /9 - 2023-2/Aprendizado de Maquina/Lab2/Dados_Radar_Estacao_Completo_2018_2022.csv')
df_raw_data.drop(df_raw_data[df_raw_data['Tp_est'] == 0.0].index, inplace=True);

Remoção inicial de colunas da base geral (2018 a 2022). A colunas removidas foram:
- Unnamed: 0 pois é a coluna de ids
- latitude e longitude pois possuem a mesma informação que as colunas lat e lon;
- distancia: Não agrega valor ao modelo. (*)

In [4]:
df_raw_data.drop(['Unnamed: 0', 'latitude', 'longitude', 'distancia'], axis=1, inplace=True)

Separando o raw data entre treinamento e teste. Anos de 2018 a 2021 para treinamento e 2022 para teste.


In [5]:
raw_train_test_group = df_raw_data.groupby(df_raw_data['time'].str.contains('2022'))

df_raw_train = raw_train_test_group.get_group(False).copy()
df_raw_test = raw_train_test_group.get_group(True).copy()

Separando os dados de treinamento, selecionando uma porção dos dados para validação

In [6]:
dates = ['2018-01', '2018-02']

raw_train_validation_group = df_raw_train.groupby(df_raw_train['time'].str.contains('|'.join(dates)))
df_raw_train = raw_train_validation_group.get_group(False).copy()
df_raw_validation = raw_train_validation_group.get_group(True).copy()


Remoção das colunas elevation e sweep pois não possuiam valor agregado na base

Para verificar que as colunas elevation e sweep não possuiam valor agregado, foi utilizado a função describe do pandas. Com esta função, foi possível verificar que ambas possuiam média, minimo e maximo identicos, além de um desvio padrão igual a 0, ou seja, todas as linhas possuiam o mesmo valor.

In [7]:
print(df_raw_train.describe()['elevation'], end="\n\n")
print(df_raw_train.describe()['sweep'])

df_raw_train.drop(['elevation', 'sweep'], axis=1, inplace=True)
df_raw_validation.drop(['elevation', 'sweep'], axis=1, inplace=True)
df_raw_test.drop(['elevation', 'sweep'], axis=1, inplace=True)

count    83083.0
mean         0.5
std          0.0
min          0.5
25%          0.5
50%          0.5
75%          0.5
max          0.5
Name: elevation, dtype: float64

count    83083.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: sweep, dtype: float64


### 2° Lista para tratamento de dados faltantes

Resultados:
- Construção da lista *input_values* que vai ser utilizada para fazer todos os tratamentos de dados faltantes.

A ideia é utilizar apenas dados originais da base para realizar o input.


Gerando a df_train_no_na, uma copia da base de treinamento apenas com os as linhas sem NaN

In [ ]:
df_train_no_na = df_raw_train.dropna()

Construindo a lista *input_values*

In [ ]:
input_values = []
columns = list(df_train_no_na.head())
for row in df_train_no_na.iterrows():
    timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    row_info = {}
    for col in columns:
        row_info[col] = row[1][col]
    row_info["time"] = timestamp

    input_values.append(row_info)

input_values = sorted(input_values, key=lambda d: d["time"])

### 3° Imputação de dados na base de treinamento
Media de 13min ao rodar no Google Colab

Resultados:
- Base de treinamento com todas as linhas com valores preenchidos (*df_train*)

Observações:
Foi tentado utilizar o KNNImputation da Sklearn, porem não encontrei uma maneira de utilizar o horário com parametro para o calculo das distancias, e da mesmo forma, realizando alguns testes não consegui utilizar uma outra base de dados como fonte dos valores para inputação (no caso, utilizar a base so com os valores em NA como feito aqui)

In [ ]:
df_train = df_raw_train.copy()

Realizando o input na base de dados de treinamento

In [ ]:
def get_fields_to_input(row, columns):
    fields = []
    for col in columns:
        if str(row[col]) == 'nan':
            fields.append(col)
    return fields

def get_data(timestamp, input_values):
    data = None
    min_distance = math.inf
    for value in input_values:
        distance = abs(timestamp - value["time"])
        if distance < min_distance:
            min_distance = distance
            data = value

    return data

columns = list(df_train.head())
for row in df_train.iterrows():
    timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    fields_to_input = get_fields_to_input(row[1], columns)
    if len(fields_to_input) > 0:
        input_data = get_data(timestamp, input_values)
        for field in fields_to_input:
            if str(row[1][field]) == 'nan':
                df_train.loc[row[0], field] = input_data[field]

### Remoção de outliers

In [ ]:
data = { '0 - 2.4': 0, '2.5 - 7.4': 0, '7.5 - 12.4': 0, '12.5 - 17.4': 0, '17.5 - 22.4': 0, '22.5 - 27.4': 0, '27.5 - 32.4': 0,  '32.4 - 37.5': 0}
for row in df_train.iterrows():
    if float(row[1]['Tp_est']) < 2.4:
        data['0 - 2.4'] += 1
    elif float(row[1]['Tp_est']) < 7.4:
        data['2.5 - 7.4'] += 1
    elif float(row[1]['Tp_est']) < 12.4:
        data['7.5 - 12.4'] += 1
    elif float(row[1]['Tp_est']) < 17.4:
        data['12.5 - 17.4'] += 1
    elif float(row[1]['Tp_est']) < 22.4:
        data['17.5 - 22.4'] += 1
    elif float(row[1]['Tp_est']) < 27.4:
        data['22.5 - 27.4'] += 1
    elif float(row[1]['Tp_est']) < 32.4:
        data['27.5 - 32.4'] += 1
    else:
        data['32.4 - 37.5'] += 1

data

{'0 - 2.4': 76345,
 '2.5 - 7.4': 7250,
 '7.5 - 12.4': 1263,
 '12.5 - 17.4': 408,
 '17.5 - 22.4': 121,
 '22.5 - 27.4': 42,
 '27.5 - 32.4': 11,
 '32.4 - 37.5': 1}

In [ ]:
df_train.drop(df_train[df_train['Tp_est'] >= 22.4].index, inplace=True)

### Oversampling

In [ ]:
# teste = df_train.drop(['Est', 'time'], axis=1)
teste = smogn.smoter(data=df_train.reset_index(drop=True), y="Tp_est", samp_method='balance')
teste

/usr/local/lib/python3.10/dist-packages/smogn/over_sampling.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = pd.Categorical(pd.factorize(
/usr/local/lib/python3.10/dist-packages/smogn/over_sampling.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, j] = pd.Categorical(pd.factorize(
dist_matrix:   0%|          | 5/17392 [00:53<51:54:13, 10.75s/it]


KeyboardInterrupt: ignored

### 4° Preparação da base de validação
Realizar a imputaçao de dados na base de validação

Resultados:
- Base de validação com todas as linhas com valores preenchidos (*df_validation*)

In [ ]:
df_validation = df_raw_validation.copy()

Realizando o input na base de dados de validação

In [ ]:
def get_fields_to_input(row, columns):
    fields = []
    for col in columns:
        if str(row[col]) == 'nan':
            fields.append(col)
    return fields

def get_data(timestamp, input_values):
    data = None
    min_distance = math.inf
    for value in input_values:
        distance = abs(timestamp - value["time"])
        if distance < min_distance:
            min_distance = distance
            data = value

    return data

columns = list(df_validation.head())
for row in df_validation.iterrows():
    timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    fields_to_input = get_fields_to_input(row[1], columns)
    if len(fields_to_input) > 0:
        input_data = get_data(timestamp, input_values)
        for field in fields_to_input:
            if str(row[1][field]) == 'nan':
                df_validation.loc[row[0], field] = input_data[col]

### Validaçao dos modelos de regressão

In [ ]:
df_train_for_validation = df_train.drop(['x', 'y', 'z', 'time'], axis=1)
df_validation_for_train = df_validation.drop(['x', 'y', 'z', 'time','Est'], axis=1)

In [ ]:
def get_model(model):
    match model:
        case 'knn':
            return neighbors.KNeighborsRegressor(n_neighbors=100)
        case 'svr':
            return svm.SVR()
        case 'linear_regression':
            return linear_model.LinearRegression()
        case 'tree_regression':
            return tree.DecisionTreeRegressor()

models = ['knn', 'svr', 'linear_regression', 'tree_regression']
est_preds = {}

validate_list = np.array(df_validation_for_train.values.tolist())
x_validate = validate_list[:, :-1]
y_validate = validate_list[:, -1]

grouped_by_est = df_train_for_validation.groupby(['Est'])
for est in grouped_by_est.groups.keys():
    group = grouped_by_est.get_group(est).copy()
    group = group.drop(['Est'], axis=1)
    train_list = np.array(group.values.tolist())
    x_train = train_list[:, :-1]
    y_train = train_list[:, -1]

    preds = {}
    for model_type in models:
        model = get_model(model_type)
        model.fit(x_train, y_train)

        pred = model.predict(x_validate)
        preds[model_type] = pred

    est_preds[est] = preds

In [ ]:
errors = {}
for est in est_preds:
    est_errors = {}
    for model in est_preds[est]:
        mse = mean_squared_error(y_validate, est_preds[est][model])
        mae = mean_absolute_error(y_validate, est_preds[est][model])
        est_errors[model] = { 'mse': mse, 'mae': mae }
    errors[est] = est_errors

In [ ]:
df_comparison = pd.DataFrame(columns=['Estação', 'Melhor Modelo MSE', 'MSE', 'Melhor Modelo MAE', 'MAE'])

for est in errors:
    min_mse = None
    min_mse_value = math.inf
    min_mae = None
    min_mae_value = math.inf

    for model in errors[est]:
        if errors[est][model]['mse'] < min_mse_value:
            min_mse = model
            min_mse_value = errors[est][model]['mse']
        if errors[est][model]['mae'] < min_mae_value:
            min_mae = model
            min_mae_value = errors[est][model]['mae']
    new_row = [est, min_mse, min_mse_value, min_mae, min_mae_value]
    df_comparison.loc[len(df_comparison)] = new_row

df_comparison

,Estação,Melhor Modelo MSE,MSE,Melhor Modelo MAE,MAE
0,Aguas_do_Vere,linear_regression,6.362265,svr,1.066952
1,Altonia,linear_regression,6.366422,svr,1.058301
2,Assis_Chateaubriand,knn,6.374977,svr,1.058298
3,Baixo_Iguacu,linear_regression,6.313777,svr,1.058299
4,Bela_Vista_Jusante,knn,6.539395,svr,1.058302
5,Boa_Vista_da_Aparecida,linear_regression,6.328819,svr,1.058298
6,Campo_Mourao,linear_regression,6.353003,svr,1.058299
7,Cascavel,knn,6.426417,svr,1.056864
8,Coronel_Domingos_Soares,linear_regression,6.363120,svr,1.066961
9,Derivacao_do_Rio_Jordao,knn,6.474518,svr,1.058302
